In [59]:
# 의사록(MPB)

import requests
from bs4 import BeautifulSoup

for i in range(1, 45):
    b_url = "https://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761"
    params = {
        'pageIndex' : i
    }

    response = requests.get(b_url, params=params)
    soup = BeautifulSoup(response.text)

    li_list = []
    li = soup.find('div', class_='bdLine type2').find('ul').find('li')
    li_li = li.find_next_siblings('li', li)
    li_list.append(li)
    li_list.extend(li_li)

    print(i)

    for x in range(0, len(li_list)):
        try:
            link = li_list[x].find('div', class_='fileGoupBox').find('ul').find('li').find('a').attrs['href']
            title = li_list[x].find('div', class_='row').find('span').find('a').find('span').find('span').text
            url2 = 'http://www.bok.or.kr' + link
            file_res = requests.get(url2)

            with open('{}.hwp'.format(title), 'wb') as f:
                f.write(file_res.text)
        except:
            link2 = li_list[x].find('div').find('div').find('a').attrs['href']
            title = li_list[x].find('div', class_='row').find('span').find('a').find('span').find('span').text
            url3 = 'http://www.bok.or.kr' + link2
            file_res2 = requests.get(url3)

            with open('{}.hwp'.format(title), 'wb') as f:
                f.write(file_res.text)
            



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44


In [ ]:
# 의사록(MPB) _종혁님_

# MPB minutes pdf download code
import requests, urllib, os
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm


# 각자 파일 경로
my_local_pdf_dir_path = 'secret_MPB_minutes'
# 디렉토리 생성
os.makedirs(f'./{my_local_pdf_dir_path}', exist_ok=True)
# b_url: base url, {page}: page number
b_url = "http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex={page}"

for i in tqdm(range(1, 40)):
    response = requests.get(b_url.format(i))
    soup = BeautifulSoup(response.text)

    # file_name_list: 고유한 파일명을 위한 파일명 리스트 추출
    file_name_list = soup.find_all('span', class_='col m10 s10 x9 ctBx')
    # tmp: 크롤링할 파일 url 목록 추출
    tmp = soup.find_all('div', class_='fileGoupBox')
    
    # 2개의 리스트를 이용하기에 range 사용
    for j in range(len(tmp)):
        # 현재 file의 파일명 할당
        file_name = file_name_list[j].find('span', class_='titlesub').getText().strip()
        # 현재 file 위해 사용할 태그
        for_download = tmp[j]
        # naem_: 파일 확장자 확인 위한 변수
        name_ = for_download.find('li').find('a').getText().strip()
        if name_[-4:]=='.hwp':
            url = ('http://www.bok.or.kr/'+for_download.find_all('li')[1].find('a')['href']) # hwp는 안되므로 그 다음 pdf를 다운로드
            with urllib.request.urlopen(url) as web_file, open(f'./{my_local_pdf_dir_path}/{file_name}.pdf', 'wb') as local_file:
                local_file.write(web_file.read())
        else:
            url = ('http://www.bok.or.kr/'+for_download.find('li').find('a')['href']) # pdf를 지정하고 있으므로 그대로 다운로드.
            with urllib.request.urlopen(url) as web_file, open(f'./{my_local_pdf_dir_path}/{file_name}.pdf', 'wb') as local_file:
                local_file.write(web_file.read())


In [57]:
# 채권 분석 리포트 https://finance.naver.com/research/debenture_list.naver
import time
from datetime import datetime
import requests
from bs4 import BeautifulSoup

for i in range (1, 198):
    bond_url = "https://finance.naver.com/research/debenture_list.naver"
    params = {
        'page' : i
    }
    headers = {'authority' : 'finance.naver.com',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36' }
   

    response = requests.get(bond_url, params=params, headers=headers)
    soup = BeautifulSoup(response.text)
    bond = soup.select('#contentarea_left .file')
    bond_list = []
    for x in range(len(bond)):
        if bond[x].find_previous_sibling('td').text == '유안타증권':
            try:    
                bond_list.append([soup.select("#contentarea_left .file a[href$='.pdf']")[x].attrs['href'], bond[x].find_previous_sibling('td').find_previous_sibling('td').text])
            except:
                print("통과")

    for y in bond_list:
        file_res = requests.get(y[0])
        title = y[-1]
        try:
            with open('{}.pdf'.format(title), 'wb') as f:
                f.write(file_res.text)
        except:
            print('통과')


    
            

통과
통과


1. 월별 기사수 '금리'라는 단어 포함
2. 각 신문사별 기사수
3. 채권분석데이터수

In [1]:
# NNC(NaverNewsCrawling)
import time
import datetime
import requests, os
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

date_date = pd.date_range('2012-12-30', '2023-08-31', freq='W') #2013년부터 2023년까지 뉴스데이터 크롤링
date_role = [date_date[:130],date_date[130:260],date_date[320:390],date_date[390:520],date_date[520:]]

dir_path = './secret_news_crawling'
os.makedirs(dir_path, exist_ok=True) # mkdir과 달리 이미 해당이름의 폴더가 있어도 에러가 나지 않는다.  

for date in tqdm(date_role[2]):  #자신의 번호로 인덱싱
    df = pd.DataFrame(columns =['신문사', '제목', '내용', '날짜'])
    DateStart = date.strftime('%Y-%m-%d')
    DateEnd = (date+datetime.timedelta(6)).strftime('%Y-%m-%d')
    last_url = f"https://finance.naver.com/news/news_search.naver?rcdate=&q=%B1%DD%B8%AE&x=20&y=14&sm=all.basic&pd=4&stDateStart={DateStart}&stDateEnd={DateEnd}&page=1"
    headers = {'authority' : 'finance.naver.com',
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}
    response = requests.get(last_url, headers=headers)
    soup = BeautifulSoup(response.text) #추가적인 decoding 작업이 필요한 content 대신 text변경
    last_page = soup.select('.pgRR a')[0]['href'].split('=')[-1] #마지막 페이지 가져오기
    print(last_page)
    for i in tqdm(range(1, int(last_page)+1)):   # 여기서 마지막 페이지까지 반복문
        news_url = f"https://finance.naver.com/news/news_search.naver?rcdate=&q=%B1%DD%B8%AE&x=20&y=14&sm=all.basic&pd=4&stDateStart={DateStart}&stDateEnd={DateEnd}&page={i}"
        headers = {'authority' : 'finance.naver.com',
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}
        response = requests.get(news_url, headers=headers)
        soup = BeautifulSoup(response.text)
        news = soup.select('.thumb a' )
        magazine = soup.select('.articleSummary .press') #신문사 데이터
        newsdate = soup.select('.articleSummary .wdate') # 날짜 데이터

        news_link = []
        magazine_name = []
        dates = []
        for i in range(len(news)):
            news_link.append('https://finance.naver.com' + news[i].attrs['href'].replace('amp;', ''))  #amp;제거해서 링크활성화
            magazine_name.append(magazine[i].text)
            dates.append(newsdate[i])

        total = []
        for index, j in enumerate(news_link):
            news_resp = requests.get(j, headers=headers)
            news_soup = BeautifulSoup(news_resp.text)
            try:
                title = news_soup.select('.article_info h3')[0].text.strip()
                content = news_soup.select('#content')[-1].text.strip()
            except:
                print(j)
                continue
            total.append([magazine_name[index],title, content, newsdate[index].text.lstrip()[:10]])
        
        df_temp = pd.DataFrame(total, columns = ['신문사', '제목', '내용', '날짜'])  #모든 리스트를 데이터프레임으로 전환
        df = pd.concat([df, df_temp], ignore_index = True)
    df.to_csv(f'{dir_path}/{DateStart}_{DateEnd}.csv', encoding='utf-8') # 1주일 간격으로 크롤링 데이터 저장    

  0%|          | 0/70 [00:00<?, ?it/s]

39


https://finance.naver.com/news/news_read.naver?article_id=0004177420&office_id=008&mode=search&query=금리&page=12


  1%|▏         | 1/70 [07:43<8:53:32, 463.95s/it]

58


...

https://finance.naver.com/news/news_read.naver?article_id=0002100294&office_id=002&mode=search&query=금리&page=42


https://finance.naver.com/news/news_read.naver?article_id=0004191286&office_id=015&mode=search&query=금리&page=54


https://finance.naver.com/news/news_read.naver?article_id=0002345814&office_id=119&mode=search&query=금리&page=55


 37%|███▋      | 26/70 [4:27:48<9:20:52, 764.82s/it] 

92


In [4]:
import pandas as pd
news = pd.read_csv('2023-08-20_2023-08-27.csv')
news

,Unnamed: 0,신문사,제목,내용,날짜
0,0,세계일보,자영업자 신용대출 ‘저금리 대환’,코로나 기간내 고금리 대출자31일부터 최대 2000만원까지 신종 코로나바이러스 감염...,2023-08-27
1,1,경향신문,전세가구 이자비용 67% 급증… 월 20만원 첫 돌파,2023년 2분기 21만4000원 기록1년 반 만에 두 배 넘게 뛰어자가가구와 격차...,2023-08-27
2,2,경향신문,'잭슨홀 쇼크' 피했지만…'금리 딜레마' 여전한 한은,■우리 경제 여파는강도 낮췄지만 美 초긴축 여전유가상승 등으로 물가도 불안금리인하 ...,2023-08-27
3,3,세계일보,은행권 ‘부산 주 금고’ 잡기 ‘군침’… “지역 기여도 높여 평가해야”,지자체 등 70% 내년 계약 만료13곳 모두 부산은행 주 금고 거래시중은행 견제·도...,2023-08-27
4,4,서울경제,[단독] 슈퍼리치 놀이터라고?...10만원으로도 투자가능한 국채 나온다,서민 장기저축 지원 목적10년·20년 등 장기물로 출시만기보유시 분리과세·가산금리 ...,2023-08-27
...,...,...,...,...,...
2609,2609,연합뉴스,"중국발 경기 불안에…한은, 24일 기준금리 5연속 동결할 듯","전문가 ""2%p 한미 금리차·가계대출 증가에도 경기 우려로 못 올려"" ""한은, 올해...",2023-08-20
2610,2610,연합뉴스,"G2발 금융위기 확산…코스피 2,400까지 떨어지나","위안화·원화 약세에 외국인 매도압력…""환율 1,300원대 후반 가능성""""중국 부동산...",2023-08-20
2611,2611,YTN,문제 생기면 '은행 탓'…정부 가계빚 정책 엇박자 비판↑,금리 인하 옥죌 때는 언제고… 가계빚 늘자 또다시 은행권에 관리 주문50년 만기 주...,2023-08-20
2612,2612,노컷뉴스,"뉴욕증시, 엔비디아 실적 발표·연준 잭슨홀 컨퍼런스에 초점",[파이낸셜뉴스] 뉴욕증시 흐름을 좌우할 엔비디아 분기실적 발표가 23일(현지시간...,2023-08-20


In [ ]:
# BAC(Bond Analysis Crawling)
import time
import datetime
import requests, os
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

# '채권보고서' 라는 이름으로 검색을 해서 나온 기사 가져오기


dir_path = './secret_bond_crawling'
os.makedirs(dir_path, exist_ok=True) # mkdir과 달리 이미 해당이름의 폴더가 있어도 에러가 나지 않는다.  
   
for i in tqdm(range(1, 199)):   # 여기서 마지막 페이지까지 반복문
    df = pd.DataFrame(columns =['신문사', '제목', '내용', '날짜'])
    bond_url = f"https://finance.naver.com/research/debenture_list.naver?&page={i}"
    headers = {'authority' : 'finance.naver.com',
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}
    response = requests.get(bond_url, headers=headers)
    soup = BeautifulSoup(response.text)
    bond = soup.select('.box_type_m a' )
    
    news_link = []
    magazine_name = []
    dates = []
    for i in range(len(news)):
        news_link.append('https://finance.naver.com' + news[i].attrs['href'].replace('amp;', ''))  #amp;제거해서 링크활성화
        magazine_name.append(magazine[i].text)
        dates.append(newsdate[i])

    total = []
    for index, j in enumerate(news_link):
        news_resp = requests.get(j, headers=headers)
        news_soup = BeautifulSoup(news_resp.text)
        try:
            title = news_soup.select('.article_info h3')[0].text.strip()
            content = news_soup.select('#content')[-1].text.strip()
        except:
            print(j)
            continue
        total.append([magazine_name[index],title, content, newsdate[index].text.lstrip()[:10]])
    
    df_temp = pd.DataFrame(total, columns = ['신문사', '제목', '내용', '날짜'])  #모든 리스트를 데이터프레임으로 전환
    df = pd.concat([df, df_temp], ignore_index = True)
df.to_csv(f'{dir_path}/{DateStart}_{DateEnd}.csv', encoding='utf-8') # 1주일 간격으로 크롤링 데이터 저장    


